In [1]:
import numpy as np
import librosa
import torch
from src import laion_clap

# quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)


def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)

model = laion_clap.CLAP_Module(enable_fusion=False, tmodel='roberta')
model.load_ckpt() # download the default pretrained checkpoint.

c:\Users\George\miniconda3\envs\patrec\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


REQUEST: <PreparedRequest [HEAD]>
REQUEST: <PreparedRequest [HEAD]>
REQUEST: <PreparedRequest [HEAD]>
REQUEST: <PreparedRequest [HEAD]>


c:\Users\George\miniconda3\envs\patrec\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


REQUEST: <PreparedRequest [HEAD]>
REQUEST: <PreparedRequest [HEAD]>


Some weights of CustomRobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.log_reweighting', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


REQUEST: <PreparedRequest [HEAD]>
Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
Activating gradients for text_branch.log_reweighting


In [3]:
# # Directly get audio embeddings from audio files
# audio_file = [
#     'assets/audio_original.wav'
# ]
# audio_embed = model.get_audio_embedding_from_filelist(x = audio_file, use_tensor=False)
# print(audio_embed[:,-20:])
# print(audio_embed.shape)

# # Get audio embeddings from audio data
# audio_data, _ = librosa.load(audio_file[0], sr=48000) # sample rate should be 48000
# audio_data = audio_data.reshape(1, -1) # Make it (1,T) or (N,T)
# audio_embed = model.get_audio_embedding_from_data(x = audio_data, use_tensor=False)
# print(audio_embed[:,-20:])
# print(audio_embed.shape)

# # Directly get audio embeddings from audio files, but return torch tensor
# audio_embed = model.get_audio_embedding_from_filelist(x = audio_file, use_tensor=True)
# print(audio_embed[:,-20:])
# print(audio_embed.shape)

# # Get audio embeddings from audio data
# audio_data, _ = librosa.load(audio_file[0], sr=48000) # sample rate should be 48000
# audio_data = audio_data.reshape(1, -1) # Make it (1,T) or (N,T)
# audio_data = torch.from_numpy(int16_to_float32(float32_to_int16(audio_data))).float() # quantize before send it in to the model
# audio_embed = model.get_audio_embedding_from_data(x = audio_data, use_tensor=True)
# print(audio_embed[:,-20:])
# print(audio_embed.shape)

# Get text embedings from texts
text_data = ["I love the contrastive learning", "I love the pretrain model"]
text_embed = model.get_text_embedding(text_data, use_tensor=True)
print(text_embed)
print(text_embed.shape)

# torch.save(text_embed, "roberta_test.pt")

# Are the embeddings the same as RoBERTa?
assert torch.all(torch.isclose(text_embed, torch.load("roberta_test.pt")))


# tensor([[-0.0092,  0.0250,  0.0036,  ...,  0.0106, -0.0013, -0.0652],
#         [-0.0201,  0.0314,  0.0824,  ..., -0.0968,  0.0011, -0.0711]],

tensor([[-0.0092,  0.0250,  0.0036,  ...,  0.0106, -0.0013, -0.0652],
        [-0.0201,  0.0314,  0.0824,  ..., -0.0968,  0.0011, -0.0711]],
       device='cuda:0', grad_fn=<DivBackward0>)
torch.Size([2, 512])
